# Feedem's forward looking cashflow prediction model

In [141]:
import pandas as pd

In [142]:
data = pd.read_csv('customers invoice aging.csv')

In [143]:
data = data[['customer_id', 'invoice_age']]

### Dataset explanation
- The CSV file above is a dump of all the paid customer invoices in Unit4, from start of time until now.
- Below you can see which columns I use and an extract of the data. The invoice_age column is the number of days the invoice was paid after the invoice date.

In [144]:
data

,customer_id,invoice_age
0,1002,25
1,1002,25
2,1002,25
3,1002,6
4,1002,12
...,...,...
212951,350001,15
212952,350001,15
212953,350001,15
212954,350001,1


## Data cleanup

There are quite a few invoices that have a payment date before the invoice date. I'm not sure how to explain it. (Maybe a discussion for another day?) Because my model wil be forward lookin I think this invoices will influence the model negatively. I decided to make the invoice_age 0 for these invoices to effectively say they were paid imediately. It is about 4000 invoices or 1.9% of the total invoices in the dataset.

In [152]:
data.loc[data.invoice_age < 0, 'invoice_age'] = 0

There are also quite a few invoices that are paid very late, e.g. one is paid more than 10 years later than the invoice date. I also think they are going to negatively influence the calculation I want to make, so I want them exclude of my model.

What I do next is to get the mean point of the invoice_age column as well as the standard deviation of the invoice_age column.

In [36]:
data['invoice_age'].std()

119.03021717144341

In [37]:
data['invoice_age'].mean()

60.22249197017224

Here I calculate two standard deviations away from the mean and exclude it from the dataset.

In [38]:
60 + 2 * 120

300

In [158]:
std2 = data[data['invoice_age'] < 300].copy() # 2 std deviations (60 + 2 * 120)

Here I create an age bucket column with the following buckets:
- 0 age = paid imediately
- 1 to 30 = 30 days
- 31 to 60 = 60 days
- 61 to 90 = 90 days
- 91 to 120 = 120 days
- bigger than 120 = 120+ days

We can play with the buckets sizes and ranges if it proves to be to course.

In [159]:
std2.loc[:,'age_bucket'] = pd.cut(std2['invoice_age'], [-1, 0, 30, 60, 90, 120, 9999], labels=['paid imediately', '30 days', '60 days', '90 days', '120 days', '120+ days'])

In [160]:
std2

,customer_id,invoice_age,age_bucket
0,1002,25,30 days
1,1002,25,30 days
2,1002,25,30 days
3,1002,6,30 days
4,1002,12,30 days
...,...,...,...
212951,350001,15,30 days
212952,350001,15,30 days
212953,350001,15,30 days
212954,350001,1,30 days


In [178]:
bucket_count = std2.groupby('age_bucket')['age_bucket'].count().reset_index(name='bucket_count')

In [185]:
bucket_count['bucket_perc'] = bucket_count.bucket_count / bucket_count.bucket_count.sum() * 100

For interest sake, this shows the invoices paid as a percentage of the total.

In [188]:
bucket_count

,age_bucket,bucket_count,bucket_perc
0,paid imediately,7310,3.522348
1,30 days,88069,42.436347
2,60 days,67616,32.581000
3,90 days,22432,10.808935
4,120 days,9176,4.421487
5,120+ days,12929,6.229883


In [126]:
customer_count = std2.groupby('customer_id')['customer_id'].count().reset_index(name='tot_count')

In [127]:
customer_age_count = std2.groupby(['customer_id', 'age_bucket'])['customer_id'].count().reset_index(name='age_count').copy()

This shows the total number of invoices per customer.

In [128]:
customer_count

,customer_id,tot_count
0,1002,188
1,1002A,2
2,1002B,42
3,1003,169
4,1003A,14
...,...,...
3375,3045,3
3376,3055,1
3377,3064,1
3378,3065,1


This shows the number of invoices per customer paid, with in the ageing baskets.

In [129]:
customer_age_count

,customer_id,age_bucket,age_count
0,1002,30 days,120
1,1002,60 days,51
2,1002,90 days,4
3,1002,120 days,4
4,1002,120+ days,9
...,...,...,...
8741,3045,60 days,3
8742,3055,60 days,1
8743,3064,30 days,1
8744,3065,30 days,1


In [192]:
customer_paying_history = pd.merge(customer_age_count, customer_count)

In [193]:
customer_paying_history['paying_prob'] = customer_paying_history.age_count / customer_paying_history.tot_count

This is final output of the model. It shows what is the probability that a customer pay an invoice with in the ageing bucket time frame. I will then use this model to predict and calculate probability whether an open invoice will be paid with in a 30 day time frame. 

In [194]:
customer_paying_history

,customer_id,age_bucket,age_count,tot_count,paying_prob
0,1002,30 days,120,188,0.638298
1,1002,60 days,51,188,0.271277
2,1002,90 days,4,188,0.021277
3,1002,120 days,4,188,0.021277
4,1002,120+ days,9,188,0.047872
...,...,...,...,...,...
8741,3045,60 days,3,3,1.000000
8742,3055,60 days,1,1,1.000000
8743,3064,30 days,1,1,1.000000
8744,3065,30 days,1,1,1.000000
